# Using pandas with BigQuery via pandas-gbq

In this notebook, we explore how to use pandas with BigQuery via the pandas_gbq library.

It's an open-source library that is maintained by PyData and volunteer contributors and has been around for quite some time (since 2017), so it has become quite broadly used in the industry.

Essentially, it’s a thin wrapper around the BigQuery client library (google-cloud-bigquery) that provides a simple interface for running SQL queries and uploading pandas dataframes to BigQuery. The results from these queries are parsed into a pandas.DataFrame object in which the shape and data types are derived from the source table.

Let's dive in and start using it!

## Install and import libraries

We start by installing and importing the required libraries (pandas, pandas_gbq, and numpy)

In [ ]:
! pip install --quiet pandas pandas-gbq numpy

### Restart the kernel

After you install the additional packages, restart the notebook kernel so it can find the packages.
Click OK when prompted after running the following code.

In [ ]:
# Restart kernel after installs
import os
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# (Wait for the kernel to restart before continuing...)

In [ ]:
import pandas as pd
import pandas_gbq
import numpy as np

## Define constants

Next, we define the constants to contain our project ID and the dataset ID at which we will save our data in BigQuery later.

**Replace "YOUR_PROJECT_ID" with your desired project ID.**

In [ ]:
PROJECT_ID = "YOUR_PROJECT_ID"
UPDATED_DATASET_ID = "new_york_taxi_trips.transformed_taxi_data_pandas_gbq"

## Define and run the query to load our data

In the next cell, we will run a simple SQL query to read in some data from the `New York Taxi Trips` BigQuery Public Dataset into a dataframe that we can then use in the remaining steps in this notebook. We will limit the number of records to 1000 because pandas_gbq will download the results of the query to our local environment.

In [ ]:
QUERY = "SELECT * FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2020` LIMIT 1000"

# Use pandas-gbq to load the data into a DataFrame
df = pandas_gbq.read_gbq(QUERY, project_id="still-sight-352221")

## Data exploration

Now that we've read the data into a dataframe, we can begin to explore our dataset.

### Preview the data

Let's take a look at some of the values in our dataset:

In [ ]:
df.head()

### Explore the data types 

We can use the dtypes property to explore the data types in the fields of our dataset:

In [ ]:
df.dtypes

### Summary statistics

We can use the describe() function to display some summary statistics about the fields in our dataset. This can help us to understand the scale of features in our dataset, by displaying statistics such as `count`, `min`, `max`, `mean`, and the standard deviation (`std`):

In [ ]:
df.describe()

### Explore missing values

Missing values can cause problems for many machine learning algorithms, so it's often important for data scientists to be aware of any missing values that exist in the dataset, and to address them accordingly. The code in the next cell will tell us how many missing values exist for each feature in the dataset:

In [ ]:
df.isnull().sum()

### Value counts

It's also often important to understand how many unique values each feature contains. This is referred to as the `cardinality` of a feature, where low cardinality features have a small number of unique values (e.g., binary features that are either `yes` or `no`), and high cardinality have a large number of unique values (e.g., product IDs).

Feature cardinality can be important to understand for tasks such as feature encoding and feature selection.

In [ ]:
df['passenger_count'].value_counts()

## Feature engineering

After exploring our data, we can perform any feature engineering that we believe could be important for helping our models to learn specific patterns in our dataset.

For example, we can engineer a new feature named `fare_per_mile` by diving the `fare_amount` feature by the `trip_distance` feature, and this new feature may be more useful if we want to build a model that estimates the fare for a given trip distance.

To avoid errors such as type mismatches during our division operation, we will convert all types to float.
To avoid division by zero, we replace all instances of zero in `trip_distance` with `numpy.finfo.eps` (epsilon), which is a tiny positive number. 

In [ ]:
df['fare_amount'] = df['fare_amount'].astype(float)
df['trip_distance'] = df['trip_distance'].astype(float)
df['fare_per_mile'] = df['fare_amount'] / df['trip_distance'].replace(0, np.finfo(float).eps)

## Writing data to BigQuery

After performing our feature engineering steps, we can write our updated data back to BigQuery for long term storage, reference, and analytics.

### Convert decimal types to float

The following is a step that only appears to be necessary when using `pandas_gbq` to write this specific dataset to BigQuery:

In [ ]:
# Convert all columns of type 'decimal.Decimal' to 'float'
for column in df.columns:
    if pd.api.types.is_object_dtype(df[column]):
        try:
            df[column] = df[column].astype(float)
        except (ValueError, TypeError):
            pass  # or handle non-convertible columns as needed

### Write the data

In [ ]:
df.to_gbq(f"{PROJECT_ID}.{UPDATED_DATASET_ID}") 